# Quantum Fourier Transform

The Quantum Fourier Transform (QFT) is an important subroutine to many quantum algorithms, most famously Shor's algorithm for factoring and the quantum phase estimation (QPE) algorithm for estimating the eigenvalues of a unitary operator [1, 2]. The QFT can be performed efficiently on a quantum computer, using Hadamard gates, controlled phase shift gates and swap gates.

## Reference

[1] Wikipedia: https://en.wikipedia.org/wiki/Quantum_Fourier_transform

[2] Nielsen, Michael A., Chuang, Isaac L. (2010). Quantum Computation and Quantum Information (2nd ed.). Cambridge: Cambridge University Press.

In [2]:
from notebook_plotting import plot_bitstrings_formatted
import matplotlib.pyplot as plt

%matplotlib inline

import numpy as np
from braket.circuits import Circuit
from braket.devices import LocalSimulator

from braket.experimental.algorithms.quantum_fourier_transform import (
    quantum_fourier_transform as qft_module
)

# Circuits

## 1) Quantum Fourier Transform (QFT)

![image info](./images/qft.png)

In [3]:
num_qubits = 4
qubits = list(range(num_qubits))
qubits

[0, 1, 2, 3]

### Hints:
1) $H$ is the Hadamard gate, implemented using the function $\textbf{Circuit.h(qubit)}$
2) $R_{k}$ is the controlled phase gate with $R_{k}$ = 
$\begin{pmatrix}
    1 & 0 \\
    0 & e^{i2\pi / 2^k}
\end{pmatrix}$
where the rotation angle is defined by $2\pi / 2^k$
3) $R_{k}$ is implemented using the function $\textbf{Circuit.cphaseshift(control_qubit, target_qubit)}$

In [ ]:
def quantum_fourier_transform_circuit(num_qubits: int) -> Circuit:
    """Construct a circuit object corresponding to the Quantum Fourier Transform (QFT)
    algorithm, applied to the argument qubits.  Does not use recursion to generate the QFT.

    Args:
        num_qubits (int): number of qubits on which to apply the QFT

    Returns:
        Circuit: qft circuit
    """

    qft_circ = Circuit()
    qubits = list(range(num_qubits))
    
    # Loop over the qubits
    for k in range(num_qubits):
        # First add a Hadamard gate
        qft_circ.h(qubits[k])

        # Then apply the controlled rotations, with weights (angles) defined by the distance
        # to the control qubit. Start on the qubit after qubit k, and iterate until the end.
        # When num_qubits==1, this loop does not run.
        for j in range(1, num_qubits - k):
            angle = 2 * math.pi / (2 ** (j + 1))
            qft_circ.cphaseshift(qubits[k + j], qubits[k], angle)

    # Then add SWAP gates to reverse the order of the qubits:
    for i in range(math.floor(num_qubits / 2)):
        qft_circ.swap(qubits[i], qubits[-i - 1])

    return qft_circ

## 2) Inverse QFT